In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext
sc=SparkContext(master="local[3]")
print(sc)
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("spark_airport").getOrCreate()

<SparkContext master=local[3] appName=pyspark-shell>


In [73]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql import Row
import pandas as pd

In [74]:
df= spark.read.csv('./fin.csv', header = 'true', inferSchema = 'true', sep = ','  )

In [75]:
data= spark.read.csv('./comprar_alquilar.csv', header = 'true', inferSchema = 'true', sep = ','  )

In [76]:
data.printSchema()

root
 |-- ingresos: integer (nullable = true)
 |-- gastos_comunes: integer (nullable = true)
 |-- pago_coche: integer (nullable = true)
 |-- gastos_otros: integer (nullable = true)
 |-- ahorros: integer (nullable = true)
 |-- vivienda: integer (nullable = true)
 |-- estado_civil: integer (nullable = true)
 |-- hijos: integer (nullable = true)
 |-- trabajo: integer (nullable = true)
 |-- comprar: integer (nullable = true)



In [77]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [78]:
featuresCreator = VectorAssembler(inputCols=["hijos","ingresos","vivienda"],outputCol="features" )

In [79]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",featuresCol='features',labelCol='comprar')

In [80]:
pipeline = Pipeline(stages=[featuresCreator,nb])

In [81]:
model = pipeline.fit(data)

In [82]:
pr = model.transform(df)

In [83]:
pr.select("hijos","ingresos","vivienda","prediction").show()

+-----+--------+--------+----------+
|hijos|ingresos|vivienda|prediction|
+-----+--------+--------+----------+
|    0|    2000|  200000|       0.0|
|    2|    6000|  320000|       1.0|
+-----+--------+--------+----------+

